# Audio Summary Generation using watsonX Part - 2

## 1. Importing Libraries

In [1]:
import os
from dotenv import load_dotenv
import os

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials

from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain import PromptTemplate
from langchain.document_loaders import TextLoader

## 2. Setting up the API Key and Endpoint

In [2]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

## 3. Setting up the model name and its parameters

In [3]:
model_id = "meta-llama/llama-2-70b-chat"

In [4]:
params = GenerateParams(
            decoding_method="greedy",
            max_new_tokens=300,
            min_new_tokens=100,
            temperature=0.7,
        )

## 4. Function to store open the txt file, convert it in to embeddings and store it in vector Database

In [5]:
def load_txt(): 
    pdf_name = 'transcriptios.txt'
    loaders = [TextLoader(pdf_name)]

    index = VectorstoreIndexCreator(
        embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L12-v2'), 
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    ).from_loaders(loaders)

    return index

## 5. Create the object of the function and call it

In [6]:
index = load_txt()

/Users/charanahu/VS_Code/EMEA/Audio_Summary/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 6. Deffining the PROMPT TEMPLATE

In [7]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in English:


Person emotion on IBM: """

In [8]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

## 7. Defing the chain with parameters

In [9]:
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=LangChainInterface(model=model_id, params=params, credentials=creds), chain_type="stuff", retriever=index.vectorstore.as_retriever(), chain_type_kwargs=chain_type_kwargs)

## 8. Test time

In [10]:
query = '''1. Summary of the conversation: \n
                     
        2. Emotions on organization/ company based on the conversation:'''


print(qa.run(query))



1. Summary of the conversation: IBM has announced Watson X, a platform for building AI models for businesses. It's different from chat GPT and focuses on business use cases like customer service, IT automation, cybersecurity, and governance. IBM will earn money through the subscription platform and consulting services.

2. Emotions on organization/company based on the conversation:

Positive:

* IBM is pleased to announce Watson X.
* It's going to have a huge impact on businesses.
* IBM is focused on business use cases.
* IBM has built a capability for bringing data in that is trusted and understandable.
* IBM has a center of excellence for generative AI.
* IBM can monetize through software and consulting.

Neutral:

* IBM is working closely with Washington to evaluate the best way to regulate AI.

No negative emotions are expressed towards IBM in the conversation.
